In [ ]:
#r "nuget:YahooFinanceApi;" 
#r "nuget: TALib.NETCore, 0.4.4"
#r "nuget:QuantLib" 

using YahooFinanceApi;
using QuantLib;
using TALib;


Installed Packages QuantLib, 1.0.5 TALib.NETCore, 0.4.4

In [ ]:
var history = await Yahoo.GetHistoricalAsync("AAPL", DateTime.Today.AddDays(-2020), DateTime.Now, Period.Daily);
public double[] input = new double[history.Count()];
for (int i=0; i<history.Count(); i++) { input[i]=(double)history[i].Close; }

//public  double[] input = new double[]{10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1};

double[] output = new double[input.Length];
int outBegIdx, outNbElement;
Core.Ema(input, 0, input.Length-1, output, out outBegIdx, out outNbElement, 5);

TSeries data = new();
EMA_Series ind = new(data, 5, true);
for (int i=0; i<input.Length; i++) { 
    data.Add(input[i]); 
    double taval = (i>=outBegIdx) ? output[i-outBegIdx]:double.NaN;
    //Console.Write($"{i}\t{data[i].v:f2}\t{ind[i].v:f5}\t{taval:f5}\n");
}
data.Count()

1393

In [ ]:
var history = await Yahoo.GetHistoricalAsync("AAPL", DateTime.Today.AddDays(-10), DateTime.Now, Period.Daily);
var ddata = history.Select(x => (t: x.DateTime, v: (double)x.Close));

//var baselist = history.Select(x => (t: x.DateTime, v: (double)x.Close));
//TSeries data = new();
//baselist.ToList().ForEach(q => data.Add((q)));

//history.Select(x => (t: x.DateTime, v: (double)x.Close)).ToList().GetType()
ddata

index,Item1,Item2
0,2022-03-21 00:00:00Z,165.380005
1,2022-03-22 00:00:00Z,168.820007
2,2022-03-23 00:00:00Z,170.210007
3,2022-03-24 00:00:00Z,174.070007
4,2022-03-25 00:00:00Z,174.720001
5,2022-03-28 00:00:00Z,175.600006
6,2022-03-29 00:00:00Z,178.960007
7,2022-03-30 00:00:00Z,177.666


In [ ]:
int period = 10;

//QuantLib SMA algorithm
SMA_Series e = new(data, period, false);  

// direct call to SMA from TA-LIB - with stitching NaNs in front
double[] output = new double[data.Count];
double[] nans = new double[period];
double[] ta_temp = new double[data.Count-period+1];
Array.Fill(nans, double.NaN);
Core.Sma(data.v.ToArray(), 0, data.Count-1, ta_temp, out outBegIdx, out outNbElement, period); //TA-LIB SMA method
nans.CopyTo(output,0);
ta_temp.CopyTo(output,period-1);

// extracted SMA algo from TA-LIB - with full calc of early values
TSeries ta_out = new();
int i = 0;
double periodTotal = 0;
do {
    periodTotal += data[i].v;
    double tempReal = periodTotal;
    if (i>=period) { periodTotal -= data[i-period].v; }
    ta_out.Add((data[i].t, tempReal/((i<period)?i:period)), false);
    i++;
} while (i < data.Count-1);


In [ ]:
// comparing the tail end of three results
Console.Write($"TA-LIB Direct\t  TALIB Algo\t QuantLib\n");
for (int i=data.Count-10; i<data.Count-1; i++) 
    Console.Write($"{output[i]:f2}\t\t  {ta_out[i].v:f2}\t {e[i].v:f2}\n");

Console.Write($"{output.Length}\t\t  {ta_out.Count()}\t\t {e.Count()}\n");



TA-LIB Direct	  TALIB Algo	 QuantLib
158.20		  177.67	 158.20
158.28		  177.44	 158.28
158.89		  177.66	 158.89
160.03		  178.61	 160.03
160.76		  179.89	 160.76
162.31		  181.00	 162.31
164.31		  182.62	 164.31
166.81		  184.71	 166.81
169.20		  187.54	 169.20
1393		  1392		 1393


In [ ]:
output.Length

1393